In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [94]:
train = pd.read_csv("train.csv")
train, validation = train_test_split(train, test_size=0.2) # pd.read_csv("test.csv")

In [72]:
def preprocess(data):
    # Cleaning unhelpful columns (unique strings)
    dataCleaned = data.drop(["PassengerId", "Name", "Cabin"], axis=1)
    oneHotData = dataCleaned.iloc[:, [0, 2]]
    
    # drop every non integer or boolean column so we can add back one-hot encoded ones
    dataCleaned = data.drop(["PassengerId", "Name", "Cabin", "HomePlanet", "Destination"], axis=1)
    
    # encode one-hot columns
    enc = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
    transformed = enc.fit_transform(oneHotData)
    
    # drop transported from training data
    if "Transported" in dataCleaned.columns:
        dataCleaned = dataCleaned.drop("Transported", axis=1)
    
    # return original data + one-hot encoded
    return pd.concat([transformed, dataCleaned], axis=1)

In [89]:
trainCleaned = preprocess(train)
testCleaned = preprocess(test)
label = train.iloc[:, -1]
label

5184     True
2847    False
33       True
1151     True
834      True
        ...  
4544     True
3127     True
229      True
3199     True
7595     True
Name: Transported, Length: 7823, dtype: bool

In [90]:
from sklearn import ensemble
model = ensemble.RandomForestClassifier()
model.fit(trainCleaned, label)

RandomForestClassifier()

In [97]:
output = pd.Series(model.predict(testCleaned))
output

0        True
1        True
2       False
3        True
4       False
        ...  
1734    False
1735     True
1736    False
1737    False
1738    False
Length: 1739, dtype: bool

In [96]:
label = validation.iloc[:, -1]
label = label.reset_index(drop=True)
label

0       False
1        True
2        True
3       False
4        True
        ...  
1734    False
1735    False
1736     True
1737    False
1738     True
Name: Transported, Length: 1739, dtype: bool

In [100]:
for i in range(iterations):
    train, validation = train_test_split(train, test_size=0.2)
    # 
    
    total_correct = 0
    for i in range(len(label)):
        if label[i] == output[i]:
            total_correct += 1
    print(total_correct / len(label))

0.5169637722829212


In [102]:
from sklearn.metrics import mean_absolute_error # MAE
from sklearn.metrics import mean_squared_error # MSE / RMSE R -> root
import numpy as np

# turns boolean values into 0/1
for i in range(len(label)): 
    if label[i]:
        label[i] = 0
    else:
        label[i] = 1
    if output[i]:
        output[i] = 0
    else:
        output[i] = 1
        
np.sqrt(mean_squared_error(label, output))

0.6950080774473623

In [45]:
trainCleaned = train.drop(["PassengerId", "Name", "Cabin"], axis=1)
transformed = trainCleaned.iloc[:, [0, 2]]

# drop every non integer or boolean column so we can add back one-hot encoded ones
trainCleaned = train.drop(["PassengerId", "Name", "Cabin", "HomePlanet", "Destination"], axis=1)
label = train.iloc[:, -1]
label

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [46]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

transformed = enc.fit_transform(transformed)

In [47]:
transformed.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [37]:
transformed = pd.concat([transformed, trainCleaned], axis=1)
transformed.head()

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [38]:
from sklearn import ensemble
model = ensemble.RandomForestClassifier()
model.fit(trainCleaned, label)

RandomForestClassifier()

In [39]:
testCleaned = test.iloc[:, [2, 5, 6, 7, 8, 9, 10, 11]]
model.predict(testCleaned)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Transported
